In [1]:
from Paciente import Paciente

In [2]:
import numpy as np
from scipy.ndimage import label, center_of_mass, sum  

In [3]:
# Se instancia la clase para el paciente numero 29
paciente29 = Paciente(29, 3)

No damaged files


Construyendo Mascaras: 100%|██████████| 348/348 [00:16<00:00, 21.32it/s]


In [7]:
pat_height = paciente29.altura
pat_weight = paciente29.peso
spacing = paciente29.spacing
data = paciente29.mascaraGeneral

pat_BSA = np.sqrt((pat_height*pat_weight)/3600) #m^2? 

# Separate the different lesions and give them different labels
# Not connected regions will be seen as different objects 
labeled_data, num_features = label(data)
print('Number of lesions detected: ' + str(num_features))

Number of lesions detected: 19


In [ ]:
# Select only the segmented data points (nonzero points)
seg = np.argwhere(labeled_data)

# Calculate centroids/centers of mass for the different lesions which are 
# represented as different objects/features in the labeled data array
centr = np.asarray(center_of_mass(data, labels = labeled_data,
                                  index = list(range(1, num_features+1))))
    
#%% Define functions (for weighted distances in general, distances on a given array,
# distances between one given point and all the others)

# Function for weighted distances
def weighted_distance(p1, p2, weights, type_of_distance):
    '''
    The function calculates the weighted distance between two points. 
    The type of distance can be selected. 
    
    Parameters
    ----------
    p1 : array_like
        Coordinates of the first point. 
    
    p2 : array_like
        Coordinates of the second point. 
    
    weights : array_like 
        Array which contains the parameters which should be used as weights to 
        calculate the weighted distance. Should be of the same length as there are
        dimensions in the data points. In case of images, these weights equal 
        the spacing of the image data in the different dimensions.
            
    type_of_distance : str 
        Type of distance to be calculated: 'euclidian', 'manhattan' or 'cityblock',
        or 'chebyshev'.

    Returns
    -------
    Distance between the two points p1 and p2.

    '''
    # First: Check if enough weight parameters are given. Follow through with 
    # the code if this is the case. If not, raise Exception and print statement. 
    try: 
        len(weights) == len(p1) == len(p2)
    except: 
        print('Error: Dimension mismatch between data points and weight array') 
    else: 
        # For every type of distance there is a formula for a weighted distance
        # which will be executed if the corresponding type is selected. 
        
        q = p1 - p2 
        
        if type_of_distance == 'euclidian':
            # Weighted Euclidian distance
            return np.sqrt(((weights * q)**2).sum())
        
        elif type_of_distance == 'manhattan' or type_of_distance == 'cityblock':
            # Weighted Manhattan or Cityblock distance 
            return (weights * abs(q)).sum()
            
        elif type_of_distance == 'chebyshev':
            # Weighted Chebyshev distance 
            return max(weights * abs(q))
            
        else: 
            # If a different string than the possible distance types is given, 
            # the following statement is printed. 
            print("Error: The selected distance type is not available. \
                  Try 'euclidian', 'manhattan' or 'cityblock', or 'chebyshev'.")


# Define function which calculates weighted distances (all three types) on a 
# given array (segmentation, either whole body lesions or only centroids)
def distances(segmentation, weights):
    '''
    The function calculates the distances between all pairs of data points 
    given by the segmentation array.
    The distances are returned in an array (nx3) as well. 

    Parameters
    ----------
    segmentation : array_like
        Segmented image data, either whole lesions or only centroids.
        Depending on the type of segmentation, one of the two following distances
        is calculated: 
            
            (a) Distances between all pairs of voxels, regardless of which 
            lesion they belong to. The segmentation should therefore contain 
            the whole lesion volume. 
            (b) Distances between pairs of centroids. The segmentation should 
            contain only centroids. 
    
    weights : array_like 
        Array which contains the parameters which should be used as weights to 
        calculate the weighted distance. Should be of the same length as there are
        dimensions in the data points. In case of images, these weights equal 
        the spacing of the image data in the different dimensions.

    Returns
    -------
    Three arrays of length n, containing the calculated distances. Each array 
    contains a different type of distance, n being the number of pairs of points.
    One array containing the indices of data points in seg, for the centroids, 
    this represents the index of the lesion. 
    
    '''
    euc_dist = [weighted_distance(segmentation[i], segmentation[j], weights, 'euclidian')
                for i in range(len(segmentation)) for j in range(len(segmentation)) if i != j]
    man_dist = [weighted_distance(segmentation[i], segmentation[j], weights, 'manhattan')
                for i in range(len(segmentation)) for j in range(len(segmentation)) if i != j]
    che_dist = [weighted_distance(segmentation[i], segmentation[j], weights, 'chebyshev')
                for i in range(len(segmentation)) for j in range(len(segmentation)) if i != j]
    index_i = [i for i in range(len(segmentation)) for j in range(len(segmentation)) if i != j]
 
    return euc_dist, man_dist, che_dist, index_i

# Function that gives back all the euclidian distances between one specific 
# lesion and all the others 
def lesion_distances(index): 
    '''
    Function that gives back all the euclidian distances between one specific
    lesion and all the others.

    Parameters
    ----------
    index : int
        Python index of the specific lesion that should be the 'central' lesion.

    Returns
    -------
    Array of distances from one specific lesion.

    '''
    # Give back the array of euclidian distances between one specific lesion and the others
    d = dist_ctr[0, np.where(dist_ctr[3] == index)[0]]
    
    return d
    
#%% Apply functions to data 
    # IF NECESSARY COMMENT OUT THE PART FOR THE DISTANCES BETWEEN VOXEL PAIRS
    
# Distances between all pairs of voxels are calculated 
euc_dist_vox, man_dist_vox, che_dist_vox, index_i_vox = distances(seg, spacing)
dist_vox = np.array([euc_dist_vox, man_dist_vox, che_dist_vox])
np.savetxt(results_dir + '/distances_vox.csv', dist_vox, delimiter = ',') 

# Distances between all pairs of centroids are calculated 
euc_dist_ctr, man_dist_ctr, che_dist_ctr, index_i_ctr = distances(centr, spacing)
dist_ctr = np.array([euc_dist_ctr, man_dist_ctr, che_dist_ctr, index_i_ctr])
np.savetxt(results_dir + '/distances_ctr.csv', dist_ctr, delimiter = ',') 

#%% Calculate parameters

# Calculation of parameters, based on the voxel distances
# Dmax: Max. distance between the two voxels that are the farthest away.
Dmax = max(euc_dist_vox)

# SDmax_euc_vox: Max. euclidian distance, normalized by BSA.
SDmax_euc_vox = Dmax/pat_BSA

# SDmax_man_vox: Max. Manhattan distance, normalized by BSA.
SDmax_man_vox = max(man_dist_vox)/pat_BSA

# SDmax_che_vox: Max. Chebyshev distance, normalized by BSA.
SDmax_che_vox = max(che_dist_vox)/pat_BSA


# Calculation of parameters, based on centroid distances 
# Dmax_patient: (Euclidian) Distance between the two lesions that are the farthest away from each other. 
Dmax_patient = max(euc_dist_ctr)

# Dmax_bulk: (Euclidian) Distance between the largest lesion and the one the farthest away from it. 
# Find largest lesion first 
sizes_of_lesions = [sum(data, labeled_data, index = i)
                    for i in range(1, num_features+1)]
largest_size = max(sizes_of_lesions) # how many pixels does the largest lesion have?
largest_index = np.where(sizes_of_lesions == largest_size)[0][0] # which Python Index does the largest lesion have?
largest_label = largest_index + 1 # which Label does the largest lesion have?  
#print('The largest lesion has label index ' + str(largest_label))
# Calculate the parameter
Dmax_bulk = max(lesion_distances(largest_index))
   
# SPREAD_bulk: Sum of the euclidian distances between the largest lesion and all the other lesions.
Spread_bulk = np.sum(lesion_distances(largest_index))

# SPREAD_patient: Over all lesions, the maximum of the sum of distances from one lesion to all the others.
sums = [np.sum(lesion_distances(i)) for i in range(num_features)]
Spread_patient = max(sums)
  
# SDmax_euc: Dmax_patient, normalized by BSA
SDmax_euc = Dmax_patient/pat_BSA

# SDmax_man: Maximum Manhattan distance between lesions, normalized by BSA. 
SDmax_man = max(man_dist_ctr)/pat_BSA

# SDmax_che: Maximum Chebyshev distance between lesions, normalized by BSA. 
SDmax_che = max(che_dist_ctr)/pat_BSA

#%% Add important features to Results file (output)
pat_features = pat_features.assign(Height = [pat_height],
                                   Weight = [pat_weight], 
                                   BSA = [pat_BSA],
                                   Dmax = [Dmax], 
                                   SDmax_euc_vox = [SDmax_euc_vox],
                                   SDmax_man_vox = [SDmax_man_vox], 
                                   SDmax_che_vox = [SDmax_che_vox], 
                                   Dmax_patient = [Dmax_patient], 
                                   Dmax_bulk = [Dmax_bulk], 
                                   Spread_bulk = [Spread_bulk],
                                   Spread_patient = [Spread_patient],
                                   SDmax_euc =[SDmax_euc],
                                   SDmax_man = [SDmax_man],
                                   SDmax_che = [SDmax_che])

pat_features.to_excel(results_dir + '/Results_Patient.xlsx')

t1 = time.time()
total = t1-t0
print('Patient ' + pat + ' successfully added.')
print('Total time calculating distances for patient ' + pat + ': ' + str(total) + 's')